In [1]:
import requests
import matplotlib.pyplot as plt
import pandas as pd
import mpld3
from mpld3 import plugins


/Users/noma/OneDrive_UTA/OneDrive - UT Arlington/mac/WorkSpace/FG-CRYPTO/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
def get_fear_greed_data(limit=30):
    url = f"https://api.alternative.me/fng/?limit={limit}&format=json&date_format=us"
    response = requests.get(url)
    data = response.json()["data"]
    # Convert the data into a DataFrame
    fear_greed_df = pd.DataFrame(data)
    fear_greed_df['date'] = pd.to_datetime(fear_greed_df['timestamp']).dt.date
    fear_greed_df['value'] = fear_greed_df['value'].astype(int)
    return fear_greed_df[['date','value_classification' ,'value']]

In [3]:
fear_greed_df = get_fear_greed_data(365*2) 
fear_greed_df

,date,value_classification,value
0,2024-10-06,Neutral,50
1,2024-10-05,Neutral,49
2,2024-10-04,Fear,41
3,2024-10-03,Fear,37
4,2024-10-02,Fear,42
...,...,...,...
725,2022-10-12,Extreme Fear,20
726,2022-10-11,Extreme Fear,24
727,2022-10-10,Extreme Fear,22
728,2022-10-09,Extreme Fear,22


In [15]:
import requests
import pandas as pd
from datetime import datetime, timedelta,date

# API from coinapi

def get_historical_price_data(symbol, start_date, end_date, api_key):
    base_url = "https://rest.coinapi.io/v1/exchangerate"
    url = f"{base_url}/{symbol}/USD/history"
    
    headers = {
        "X-CoinAPI-Key": api_key
    }
    
    params = {
        "period_id": "1DAY",
        "time_start": start_date.isoformat(),
        "time_end": end_date.isoformat(),
        "limit": 10000
    }
    
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    
    if not isinstance(data, list) or len(data) == 0:
        print(f"Unexpected API response: {data}")
        return pd.DataFrame(columns=['date', 'daily_min_price', 'daily_max_price', 'daily_avg_price'])
    
    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['time_period_start']).dt.date
    df = df.rename(columns={'rate_open': 'daily_min_price', 'rate_high': 'daily_max_price', 'rate_close': 'daily_avg_price'})
    return df[['date', 'daily_min_price', 'daily_max_price', 'daily_avg_price']]


# Replace with your actual CoinAPI key
api_key = "0619394E-1016-4BB5-BADC-558EE8C4878E"

def get_eth_price_data(days=30):
    end_date = date.today()
    start_date = end_date - timedelta(days=days)
    return get_historical_price_data("ETH", start_date, end_date, api_key)

def get_bitcoin_price_data(days=30):
    end_date = date.today()
    start_date = end_date - timedelta(days=days)
    return get_historical_price_data("BTC", start_date, end_date, api_key)

#Usage example:

# bitcoin_df = get_bitcoin_price_data(time_period)


In [11]:
def merge_data(fear_greed_df, eth_df):
    # Merge the two DataFrames on the 'timestamp' column
    merged_df = pd.merge(fear_greed_df, eth_df, on='date', how='inner')
    return merged_df

In [116]:
def plot_data_with_fear_greed_alerts(merged_df, symbol):
    # Sort the dataframe by date
    merged_df = merged_df.sort_values('date')

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add cryptocurrency price trace
    fig.add_trace(
        go.Scatter(x=merged_df['date'], y=merged_df['daily_avg_price'], name=f"{symbol} Price", line=dict(color='blue')),
        secondary_y=False,
    )

    # Add Fear & Greed Index trace
    fig.add_trace(
        go.Scatter(x=merged_df['date'], y=merged_df['value'], name="Fear & Greed Index", line=dict(color='black', width=2)),
        secondary_y=True,
    )

    # Add colored background for different index ranges
    fig.add_hrect(y0=0, y1=25, line_width=0, fillcolor="green", opacity=0.2, secondary_y=True)
    fig.add_hrect(y0=25, y1=45, line_width=0, fillcolor="orange", opacity=0.2, secondary_y=True)
    fig.add_hrect(y0=45, y1=55, line_width=0, fillcolor="yellow", opacity=0.2, secondary_y=True)
    fig.add_hrect(y0=55, y1=75, line_width=0, fillcolor="lightgreen", opacity=0.2, secondary_y=True)
    fig.add_hrect(y0=75, y1=100, line_width=0, fillcolor="red", opacity=0.2, secondary_y=True)

    # Add alert zones
    fig.add_hrect(y0=0, y1=20, line_width=0, fillcolor="green", opacity=0.2, secondary_y=True)
    fig.add_hrect(y0=80, y1=100, line_width=0, fillcolor="red", opacity=0.2, secondary_y=True)

    # Add annotations for alert zones
    fig.add_annotation(x=merged_df['date'].iloc[-1], y=10, text="Extreme Fear Zone", showarrow=False, font=dict(color="green"), yref="y2")
    fig.add_annotation(x=merged_df['date'].iloc[-1], y=90, text="Extreme Greed Zone", showarrow=False, font=dict(color="red"), yref="y2")

    # Update layout
    fig.update_layout(
        title_text=f"Crypto Fear & Greed Index vs {symbol} Price with Alert Zones",
        xaxis_title="Date",
        yaxis_title=f"{symbol} Price (USD)",
        yaxis2_title="Fear & Greed Index",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        hovermode="x unified"
    )

    # Update y-axes ranges
    fig.update_yaxes(range=[merged_df['daily_avg_price'].min() * 0.8, merged_df['daily_avg_price'].max() * 1.2], secondary_y=False)
    fig.update_yaxes(range=[0, 100], tickvals=[0, 20, 40, 60, 80, 100],
                     ticktext=['0<br>Extreme<br>Fear', '20', '40', '60', '80', '100<br>Extreme<br>Greed'],
                     secondary_y=True)

    return fig



In [117]:
time_period = 365*2
bitcoin_df = get_bitcoin_price_data(time_period)  # Last 120 days

merged_bitcoin_fg_df = merge_data(fear_greed_df, bitcoin_df)
fig = plot_data_with_fear_greed_alerts(merged_bitcoin_fg_df, "BTC")
fig.show()


In [118]:
eth_df = get_eth_price_data(time_period)  # Last 120 days
merged_eth_fg_df = merge_data(fear_greed_df, eth_df)
fig = plot_data_with_fear_greed_alerts(merged_eth_fg_df, "ETH")
fig.show()


In [131]:
def trading_strategy_buy_and_hold(symbol,df,investment_amount):
    # Initialize variables
    df = df.sort_values('date')
    held = 0
    total_investment = 0
    trades = []
    # df['MA'] = df['daily_avg_price'].rolling(window=60).mean()
    
    for index, row in df.iterrows():
        date = row['date']
        price = row['daily_avg_price']
        index = row['value']
        sentiment = row['value_classification']
        # ma = row['MA']

        if sentiment == 'Extreme Fear':
            # Buy Bitcoin
            usd_balance = investment_amount 
            if index < 20:
                usd_balance += 500
                
            to_buy = usd_balance / price
            held += to_buy
            trades.append(f"Buy: {date}, usd_balance: ${usd_balance:.2f}, Price: ${price:.2f}, {symbol}: {to_buy:.6f}, Sentiment: {index}, {sentiment}")
            total_investment += usd_balance
        

    # Calculate final portfolio value
    final_value = (held * df.iloc[-1]['daily_avg_price'])

    return trades, held,final_value,total_investment

# Run the strategy
symbol = "BTC"
investment_amount = 1000
trades, held,final_value,total_investment = trading_strategy_buy_and_hold(symbol,merged_bitcoin_fg_df,investment_amount)

# Print results
print(f"{symbol}: only buy when sentiment is Extreme Fear")
print(f"Number of trades: {len(trades)}")
print(f"Bought: {held:.6f} {symbol}")
print(f"final_btc_price: ${merged_bitcoin_fg_df.sort_values('date').iloc[-1]['daily_avg_price']:.2f}")
print(f"Total investment: ${total_investment:.2f}")
print(f"Final portfolio value: ${final_value:.2f}")
print(f"Profit: ${final_value - total_investment:.2f}")
annualized_return = ((final_value / total_investment) ** (365 / len(merged_bitcoin_fg_df))) - 1
print(f"Annualized Return: {annualized_return * 100:.2f}%")
print("\nTrade History:")
for trade in trades:
    print(trade)



BTC: only buy when sentiment is Extreme Fear
Number of trades: 290
Bought: 12.363675 BTC
final_btc_price: $62056.42
Total investment: $337500.00
Final portfolio value: $767245.44
Profit: $429745.44
Annualized Return: 22.81%

Trade History:
Buy: 2021-05-16, usd_balance: $1000.00, Price: $46452.90, BTC: 0.021527, Sentiment: 20, Extreme Fear
Buy: 2021-05-18, usd_balance: $1000.00, Price: $42891.97, BTC: 0.023314, Sentiment: 21, Extreme Fear
Buy: 2021-05-19, usd_balance: $1000.00, Price: $36744.67, BTC: 0.027215, Sentiment: 23, Extreme Fear
Buy: 2021-05-20, usd_balance: $1500.00, Price: $40610.33, BTC: 0.036936, Sentiment: 11, Extreme Fear
Buy: 2021-05-21, usd_balance: $1500.00, Price: $37322.78, BTC: 0.040190, Sentiment: 19, Extreme Fear
Buy: 2021-05-22, usd_balance: $1500.00, Price: $37525.34, BTC: 0.039973, Sentiment: 12, Extreme Fear
Buy: 2021-05-23, usd_balance: $1500.00, Price: $34728.05, BTC: 0.043193, Sentiment: 14, Extreme Fear
Buy: 2021-05-24, usd_balance: $1500.00, Price: $38835

In [132]:
symbol = "ETH"
investment_amount = 1000
trades, held,final_value,total_investment = trading_strategy_buy_and_hold(symbol,merged_eth_fg_df,investment_amount)

# Print results
print(f"{symbol}: only buy when sentiment is Extreme Fear")
print(f"Number of trades: {len(trades)}")
print(f"Bought: {held:.6f} {symbol}")
print(f"final_eth_price: ${merged_eth_fg_df.sort_values('date').iloc[-1]['daily_avg_price']:.2f}")
print(f"Total investment: ${total_investment:.2f}")
print(f"Final portfolio value: ${final_value:.2f}")
print(f"Profit: ${final_value - total_investment:.2f}")
annualized_return = ((final_value / total_investment) ** (365 / len(merged_eth_fg_df))) - 1
print(f"Annualized Return: {annualized_return * 100:.2f}%")
print("\nTrade History:")
for trade in trades:
    print(trade)


ETH: only buy when sentiment is Extreme Fear
Number of trades: 290
Bought: 188.977399 ETH
final_eth_price: $2414.56
Total investment: $337500.00
Final portfolio value: $456296.45
Profit: $118796.45
Annualized Return: 7.84%

Trade History:
Buy: 2021-05-16, usd_balance: $1000.00, Price: $3584.54, ETH: 0.278976, Sentiment: 20, Extreme Fear
Buy: 2021-05-18, usd_balance: $1000.00, Price: $3376.88, ETH: 0.296131, Sentiment: 21, Extreme Fear
Buy: 2021-05-19, usd_balance: $1000.00, Price: $2442.99, ETH: 0.409334, Sentiment: 23, Extreme Fear
Buy: 2021-05-20, usd_balance: $1500.00, Price: $2774.10, ETH: 0.540715, Sentiment: 11, Extreme Fear
Buy: 2021-05-21, usd_balance: $1500.00, Price: $2434.74, ETH: 0.616081, Sentiment: 19, Extreme Fear
Buy: 2021-05-22, usd_balance: $1500.00, Price: $2298.79, ETH: 0.652518, Sentiment: 12, Extreme Fear
Buy: 2021-05-23, usd_balance: $1500.00, Price: $2099.67, ETH: 0.714398, Sentiment: 14, Extreme Fear
Buy: 2021-05-24, usd_balance: $1500.00, Price: $2650.79, ETH:

In [106]:
def trading_strategy_buy_and_hold(symbol,df,investment_amount):
    # Initialize variables
    df = df.sort_values('date')
    held = 0
    total_investment = 0
    trades = []
    # df['MA'] = df['daily_avg_price'].rolling(window=60).mean()
    
    for index, row in df.iterrows():
        date = row['date']
        price = row['daily_avg_price']
        index = row['value']
        sentiment = row['value_classification']
        # ma = row['MA']

        if sentiment == 'Extreme Fear':
            # Buy Bitcoin
            usd_balance = investment_amount 
            if index < 20:
                usd_balance += 500
                
            to_buy = usd_balance / price
            held += to_buy
            trades.append(f"Buy: {date}, usd_balance: ${usd_balance:.2f}, Price: ${price:.2f}, {symbol}: {to_buy:.6f}, Sentiment: {index}, {sentiment}")
            total_investment += usd_balance
        

    # Calculate final portfolio value
    final_value = (held * df.iloc[-1]['daily_avg_price'])

    return trades, held,final_value,total_investment

# Run the strategy
symbol = "BTC"
investment_amount = 1000
trades, held,final_value,total_investment = trading_strategy_buy_and_hold(symbol,merged_bitcoin_fg_df,investment_amount)

# Print results
print(f"{symbol}: only buy when sentiment is Extreme Fear")
print(f"Number of trades: {len(trades)}")
print(f"Bought: {held:.6f} {symbol}")
print(f"Total investment: ${total_investment:.2f}")
print(f"Final portfolio value: ${final_value:.2f}")
print(f"Profit: ${final_value - total_investment:.2f}")
annualized_return = ((final_value / total_investment) ** (365 / len(merged_bitcoin_fg_df))) - 1
print(f"Annualized Return: {annualized_return * 100:.2f}%")
print("\nTrade History:")
for trade in trades:
    print(trade)



BTC: only buy when sentiment is Extreme Fear
Number of trades: 290
Bought: 12.363675 BTC
Total investment: $337500.00
Final portfolio value: $767245.44
Profit: $429745.44
Annualized Return: 22.81%

Trade History:
Buy: 2021-05-16, usd_balance: $1000.00, Price: $46452.90, BTC: 0.021527, Sentiment: 20, Extreme Fear
Buy: 2021-05-18, usd_balance: $1000.00, Price: $42891.97, BTC: 0.023314, Sentiment: 21, Extreme Fear
Buy: 2021-05-19, usd_balance: $1000.00, Price: $36744.67, BTC: 0.027215, Sentiment: 23, Extreme Fear
Buy: 2021-05-20, usd_balance: $1500.00, Price: $40610.33, BTC: 0.036936, Sentiment: 11, Extreme Fear
Buy: 2021-05-21, usd_balance: $1500.00, Price: $37322.78, BTC: 0.040190, Sentiment: 19, Extreme Fear
Buy: 2021-05-22, usd_balance: $1500.00, Price: $37525.34, BTC: 0.039973, Sentiment: 12, Extreme Fear
Buy: 2021-05-23, usd_balance: $1500.00, Price: $34728.05, BTC: 0.043193, Sentiment: 14, Extreme Fear
Buy: 2021-05-24, usd_balance: $1500.00, Price: $38835.24, BTC: 0.038625, Sentime

In [115]:
def trading_strategy_buy_and_hold(symbol,df,investment_amount):
    # Initialize variables
    df = df.sort_values('date')
    held = 0
    total_investment = 0
    trades = []
    # df['MA'] = df['daily_avg_price'].rolling(window=60).mean()
    
    for index, row in df.iterrows():
        date = row['date']
        price = row['daily_avg_price']
        index = row['value']
        sentiment = row['value_classification']
        # ma = row['MA']

        if sentiment == 'Extreme Fear':
            # Buy Bitcoin
            usd_balance = investment_amount 
            # if index < 20:
            #     usd_balance += 500
                
            to_buy = usd_balance / price
            held += to_buy
            trades.append(f"Buy: {date}, usd_balance: ${usd_balance:.2f}, Price: ${price:.2f}, {symbol}: {to_buy:.6f}, Sentiment: {index}, {sentiment}")
            total_investment += usd_balance
        

    # Calculate final portfolio value
    final_value = (held * df.iloc[-1]['daily_avg_price'])

    return trades, held,final_value,total_investment

# Run the strategy
symbol = "BTC"
investment_amount = 1000
trades, held,final_value,total_investment = trading_strategy_buy_and_hold(symbol,merged_bitcoin_fg_df,investment_amount)

# Print results
print(f"{symbol}: only buy when sentiment is Extreme Fear")
print(f"Number of trades: {len(trades)}")
print(f"Bought: {held:.6f} {symbol}")
print(f"Total investment: ${total_investment:.2f}")
print(f"final_btc_price: ${merged_bitcoin_fg_df.sort_values('date').iloc[-1]['daily_avg_price']:.2f}")
print(f"Final portfolio value: ${final_value:.2f}")
print(f"Profit: ${final_value - total_investment:.2f}")
annualized_return = ((final_value / total_investment) ** (365 / len(merged_bitcoin_fg_df))) - 1
print(f"Annualized Return: {annualized_return * 100:.2f}%")
print("\nTrade History:")
for trade in trades:
    print(trade)



BTC: only buy when sentiment is Extreme Fear
Number of trades: 290
Bought: 10.649999 BTC
Total investment: $290000.00
final_btc_price: $62056.42
Final portfolio value: $660900.83
Profit: $370900.83
Annualized Return: 22.88%

Trade History:
Buy: 2021-05-16, usd_balance: $1000.00, Price: $46452.90, BTC: 0.021527, Sentiment: 20, Extreme Fear
Buy: 2021-05-18, usd_balance: $1000.00, Price: $42891.97, BTC: 0.023314, Sentiment: 21, Extreme Fear
Buy: 2021-05-19, usd_balance: $1000.00, Price: $36744.67, BTC: 0.027215, Sentiment: 23, Extreme Fear
Buy: 2021-05-20, usd_balance: $1000.00, Price: $40610.33, BTC: 0.024624, Sentiment: 11, Extreme Fear
Buy: 2021-05-21, usd_balance: $1000.00, Price: $37322.78, BTC: 0.026793, Sentiment: 19, Extreme Fear
Buy: 2021-05-22, usd_balance: $1000.00, Price: $37525.34, BTC: 0.026649, Sentiment: 12, Extreme Fear
Buy: 2021-05-23, usd_balance: $1000.00, Price: $34728.05, BTC: 0.028795, Sentiment: 14, Extreme Fear
Buy: 2021-05-24, usd_balance: $1000.00, Price: $38835

In [113]:
merged_bitcoin_fg_df.head()
merged_bitcoin_fg_df.sort_values('date').iloc[-1]['daily_avg_price']

np.float64(62056.4222048493)

In [98]:
def trading_strategy_buy_and_hold(symbol,df,investment_amount):
    # Initialize variables
    df = df.sort_values('date')
    held = 0
    total_investment = 0
    trades = []
    df['MA'] = df['daily_avg_price'].rolling(window=20).mean()
    
    for index, row in df.iterrows():
        date = row['date']
        price = row['daily_avg_price']
        index = row['value']
        sentiment = row['value_classification']
        ma = row['MA']

        if sentiment == 'Extreme Fear' and price < ma:
            # Buy Bitcoin
            if index < 20:
                investment_amount += 500
            usd_balance = investment_amount  # Starting with $10,000
            to_buy = usd_balance / price
            held += to_buy
            trades.append(f"Buy: {date}, Price: ${price:.2f}, {symbol}: {to_buy:.6f}, MA: {ma:.2f}, Sentiment: {index}, {sentiment}")
            total_investment += usd_balance

    # Calculate final portfolio value
    final_value = (held * df.iloc[-1]['daily_avg_price'])

    return trades, held,final_value,total_investment

# Run the strategy
symbol = "BTC"
investment_amount = 1000
trades, held,final_value,total_investment = trading_strategy_buy_and_hold(symbol,merged_bitcoin_fg_df,investment_amount)

# Print results
print(f"{symbol}: only buy when sentiment is Extreme Fear and price is less than MA")
print(f"Number of trades: {len(trades)}")
print(f"Bought: {held:.6f} {symbol}")
print(f"Total investment: ${total_investment:.2f}")
print(f"Final portfolio value: ${final_value:.2f}")
print(f"Profit: ${final_value - total_investment:.2f}")
annualized_return = ((final_value / total_investment) ** (365 / len(merged_bitcoin_fg_df))) - 1
print(f"Annualized Return: {annualized_return * 100:.2f}%")
print("\nTrade History:")
for trade in trades:
    print(trade)



BTC: only buy when sentiment is Extreme Fear and price is less than MA
Number of trades: 240
Bought: 233.783635 BTC
Total investment: $5501000.00
Final portfolio value: $14507775.94
Profit: $9006775.94
Annualized Return: 27.46%

Trade History:
Buy: 2021-05-16, Price: $46452.90, BTC: 0.021527, MA: 54511.01, Sentiment: 20, Extreme Fear
Buy: 2021-05-18, Price: $42891.97, BTC: 0.023314, MA: 53336.01, Sentiment: 21, Extreme Fear
Buy: 2021-05-19, Price: $36744.67, BTC: 0.027215, MA: 52494.93, Sentiment: 23, Extreme Fear
Buy: 2021-05-20, Price: $40610.33, BTC: 0.036936, MA: 51640.37, Sentiment: 11, Extreme Fear
Buy: 2021-05-21, Price: $37322.78, BTC: 0.053587, MA: 50615.78, Sentiment: 19, Extreme Fear
Buy: 2021-05-22, Price: $37525.34, BTC: 0.066622, MA: 49662.40, Sentiment: 12, Extreme Fear
Buy: 2021-05-23, Price: $34728.05, BTC: 0.086386, MA: 48540.11, Sentiment: 14, Extreme Fear
Buy: 2021-05-24, Price: $38835.24, BTC: 0.090124, MA: 47820.21, Sentiment: 10, Extreme Fear
Buy: 2021-05-25, Pri

In [77]:
symbol = "ETH"
investment_amount = 1000
trades, held,final_value,total_investment = trading_strategy_buy_and_hold(symbol,merged_eth_fg_df,investment_amount)

# Print results
print(f"Symbol: {symbol}")
print(f"Number of trades: {len(trades)}")
print(f"Bought: {held:.6f} {symbol}")
print(f"Total investment: ${total_investment:.2f}")
print(f"Final portfolio value: ${final_value:.2f}")
print(f"Profit: ${final_value - total_investment:.2f}")
annualized_return = ((final_value / total_investment) ** (365 / len(merged_bitcoin_fg_df))) - 1
print(f"Annualized Return: {annualized_return * 100:.2f}%")
print("\nTrade History:")
for trade in trades:
    print(trade)

Symbol: ETH
Number of trades: 257
Bought: 144.607816 ETH
Total investment: $257000.00
Final portfolio value: $349163.62
Profit: $92163.62
Annualized Return: 7.97%

Trade History:
Buy: 2021-05-19, Price: $2442.99, ETH: 0.409334, Sentiment: 23, Extreme Fear
Buy: 2021-05-21, Price: $2434.74, ETH: 0.410721, Sentiment: 19, Extreme Fear
Buy: 2021-05-22, Price: $2298.79, ETH: 0.435012, Sentiment: 12, Extreme Fear
Buy: 2021-05-23, Price: $2099.67, ETH: 0.476266, Sentiment: 14, Extreme Fear
Buy: 2021-05-28, Price: $2413.15, ETH: 0.414397, Sentiment: 21, Extreme Fear
Buy: 2021-05-29, Price: $2279.10, ETH: 0.438771, Sentiment: 18, Extreme Fear
Buy: 2021-05-30, Price: $2385.31, ETH: 0.419233, Sentiment: 10, Extreme Fear
Buy: 2021-05-31, Price: $2706.91, ETH: 0.369425, Sentiment: 18, Extreme Fear
Buy: 2021-06-01, Price: $2634.10, ETH: 0.379637, Sentiment: 20, Extreme Fear
Buy: 2021-06-02, Price: $2706.42, ETH: 0.369492, Sentiment: 23, Extreme Fear
Buy: 2021-06-05, Price: $2629.00, ETH: 0.380373, Se